### Importing Necessary Libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max

### Creating a SparkSession

In [2]:
spark = SparkSession.builder.appName("FIFA Ranking Analysis").getOrCreate()
spark

### Reading the dataset

In [3]:
data = spark.read.csv("FWC2022.csv", header=True, inferSchema=True)
data.show(10)

+-----------+---------+-----------+----+-------------+-------+---------------+
|       team|team_code|association|rank|previous_rank| points|previous_points|
+-----------+---------+-----------+----+-------------+-------+---------------+
|     Brazil|      BRA|   CONMEBOL|   1|            1| 1841.3|        1837.56|
|    Belgium|      BEL|       UEFA|   2|            2|1816.71|        1821.92|
|  Argentina|      ARG|   CONMEBOL|   3|            3|1773.88|        1770.65|
|     France|      FRA|       UEFA|   4|            4|1759.78|        1764.85|
|    England|      ENG|       UEFA|   5|            5|1728.47|        1737.46|
|      Italy|      ITA|       UEFA|   6|            7|1726.14|        1713.86|
|      Spain|      ESP|       UEFA|   7|            6|1715.22|        1716.93|
|Netherlands|      NED|       UEFA|   8|            8|1694.51|        1679.41|
|   Portugal|      POR|       UEFA|   9|            9|1676.56|        1678.65|
|    Denmark|      DEN|       UEFA|  10|           1

#### Question 1: How many teams are included in the dataset?

In [4]:
num_teams = data.count()
print("Number of teams:", num_teams)

Number of teams: 211


#### Question 2: Which team has the highest rank?

In [5]:
highest_rank_team = data.orderBy("rank").select("team").first()[0]
print("Team with the highest rank:", highest_rank_team)

Team with the highest rank: Brazil


#### Question 3: Top five teams and their points

In [6]:
top_5_teams_df = data.orderBy('points', ascending=False).select('team', 'points').limit(5)
top_5_teams_df.show()

+---------+-------+
|     team| points|
+---------+-------+
|   Brazil| 1841.3|
|  Belgium|1816.71|
|Argentina|1773.88|
|   France|1759.78|
|  England|1728.47|
+---------+-------+



#### Question 4: What is the average rank among all the teams?

In [7]:
average_rank = data.agg({"rank": "avg"}).collect()[0][0]
print("Average rank among all teams:", average_rank)

Average rank among all teams: 106.0


#### Question 5: Which team has the highest points?

In [8]:
highest_points_team = data.orderBy("points", ascending=False).select("team").first()[0]
print("Team with the highest points:", highest_points_team)

Team with the highest points: Brazil


#### Question 6: List the top 5 teams with the highest points.

In [9]:
top_5_teams = data.orderBy('points', ascending=False).select('team').limit(5).collect()
print("Top 5 Teams with Highest Points:")
[row['team'] for row in top_5_teams]

Top 5 Teams with Highest Points:


['Brazil', 'Belgium', 'Argentina', 'France', 'England']

#### Question 7: What is the total number of points across all teams?

In [10]:
total_points = data.agg({"points": "sum"}).collect()[0][0]
print("Total number of points across all teams:", total_points)

Total number of points across all teams: 257565.02999999997


#### Question 7: How many teams belong to the UEFA association?

In [11]:
uefa_teams_count = data.filter(data.association == "UEFA")
print("Number of teams belonging to UEFA association:", uefa_teams_count.count())

Number of teams belonging to UEFA association: 55


#### Question 8: Which teams belong to the CONMEBOL association

In [12]:
conmebol_teams_df = data.filter(data['association'] == 'CONMEBOL').select('team')
conmebol_teams_df.show()

+---------+
|     team|
+---------+
|   Brazil|
|Argentina|
|  Uruguay|
| Colombia|
|     Peru|
|    Chile|
|  Ecuador|
| Paraguay|
|Venezuela|
|  Bolivia|
+---------+



#### Question 9: What is the rank of the Qatars team?

In [13]:
qatars_rank = data.filter(data.team == "Qatar").select("rank").first()[0]
print("Rank of the Qatars team:", qatars_rank)

Rank of the Qatars team: 50


#### Question 10: What is the average previous points of all teams?

In [14]:
average_previous_points = data.agg({"previous_points": "avg"}).collect()[0][0]
print("Average previous points of all teams:", average_previous_points)

Average previous points of all teams: 1220.645592417061


#### Question 11: Which team has the highest difference between current and previous rank?

In [15]:

df = data.withColumn("rank_difference", data.previous_rank - data.rank)
highest_rank_difference_team = df.orderBy("rank_difference", ascending=False).select("team").first()[0]
print("Team with the highest difference between current and previous rank:", highest_rank_difference_team)

Team with the highest difference between current and previous rank: Scotland


#### Question 12: Which teams' ranks have improved from the previous ranking and with how many points?

In [16]:
rank_difference_df = data.withColumn("rank_difference", data["previous_rank"] - data["rank"])
improved_teams_rank_diff_df = rank_difference_df.filter(col("rank_difference") > 0).select("team", "rank_difference")
improved_teams_rank_diff_df.show()

+-------------+---------------+
|         team|rank_difference|
+-------------+---------------+
|        Italy|              1|
|      Croatia|              3|
|  Switzerland|              1|
|      IR Iran|              2|
|       Serbia|              4|
|      Morocco|              1|
|   Costa Rica|              3|
|       Russia|              2|
|      Hungary|              1|
|      Algeria|              4|
|    Australia|              1|
|        Egypt|              1|
|     Scotland|              5|
|       Canada|              2|
|     Paraguay|              3|
|Côte d'Ivoire|              4|
| Saudi Arabia|              2|
|      Romania|              1|
| Burkina Faso|              1|
|      Finland|              3|
+-------------+---------------+
only showing top 20 rows



#### Question 13: How many teams have a rank lower than 10?

In [17]:
teams_rank_below_10 = df.filter(df.rank < 10).count()
print("Number of teams with a rank lower than 10:", teams_rank_below_10)

Number of teams with a rank lower than 10: 9


#### Question 14: Which teams have ranks below 10

In [18]:
rank_below_10_df = data.filter(data['rank'] <= 10).select('team')
rank_below_10_df.show()

+-----------+
|       team|
+-----------+
|     Brazil|
|    Belgium|
|  Argentina|
|     France|
|    England|
|      Italy|
|      Spain|
|Netherlands|
|   Portugal|
|    Denmark|
+-----------+



#### Question 15: Which association holds highest points?

In [19]:
max_points_df = data.groupBy('association').agg(max('points').alias('max_points'))
highest_points_association_df = max_points_df.orderBy('max_points', ascending=False).limit(1)
highest_points_association = highest_points_association_df.select('association', 'max_points').first()
print("Association with the highest points:", highest_points_association['association'])
print("Points:", highest_points_association['max_points'])

Association with the highest points: CONMEBOL
Points: 1841.3
